# Video Indexer API Samples (Python)

## Contributors

- Korkrid Kyle Akepanidtaworn, AI Specialized CSA, Global Customer Success

## Overview

This notebook provides samples for the following operations in Video Indexer:   

(1) Get account details.   
(2) Upload a video from URL.   
(2A) Upload a video from local file.   
(3) Wait for the video to finish indexing.   
(4) Search for video and get insights.
(5) Use the Widgets API.   
   
Make sure you're logged-in with `az` to authenticate your account.   
   
Copy the `.env.example` file to a new file named `.env`, and update the values with your own account settings.

In [1]:
#   -*- coding: utf-8 -*-
%pip install dotenv
%pip install requests
%pip install azure-storage-blob azure-identity
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import libraries
import time  # For execution time tracking
import json # For JSON handling
import re # For regular expressions
import requests # For making HTTP requests
import sys # To interact with the Python runtime environment
from dotenv import dotenv_values # For loading environment variables from .env file
from dotenv import load_dotenv # For loading environment variables from .env file
from pprint import pprint # For pretty-printing JSON

# Import the required libraries for Azure AI VIdeo Indexer
from VideoIndexerClient.Consts import Consts
from VideoIndexerClient.VideoIndexerClient import VideoIndexerClient

# Import the required libraries for Azure Blob Storage and OpenAI
import os  
import base64
from openai import AzureOpenAI  # Interface for interacting with Azure-hosted OpenAI services

# Load environment variables from the specified .env file
# This allows sensitive keys and endpoints to be managed securely outside the code
load_dotenv(".env")

True

In [3]:
# Start execution time tracking
start_time = time.time()

# Extract required configuration parameters with validation
AccountName = os.getenv("VIDEOINDEXER_AccountName")
ResourceGroup = os.getenv("VIDEOINDEXER_ResourceGroup")
SubscriptionId = os.getenv("VIDEOINDEXER_SubscriptionId")

if not all([AccountName, ResourceGroup, SubscriptionId]):
    raise ValueError("Missing required configuration values in .env file")

# Define API version and endpoints
ApiVersion = '2024-01-01'  # Ensure this is the latest API version
ApiEndpoint = 'https://api.videoindexer.ai'
AzureResourceManager = 'https://management.azure.com'

# Initialize constants for API requests
consts = Consts(ApiVersion, ApiEndpoint, AzureResourceManager, AccountName, ResourceGroup, SubscriptionId)

# Initialize Video Indexer Client
print("Initializing Video Indexer Client...")
client = VideoIndexerClient()

# Authenticate and retrieve access tokens
try:
    print("Authenticating with Video Indexer...")
    auth_response = client.authenticate_async(consts)
    print("Authentication successful!")
    pprint(auth_response)
except Exception as e:
    print(f"Authentication failed: {e}")
    exit(1)

# Log execution time
execution_time = time.time() - start_time
print(f"Script executed in {execution_time:.2f} seconds.")

Initializing Video Indexer Client...
Authenticating with Video Indexer...
Authentication successful!
None
Script executed in 3.80 seconds.


In [4]:
# Verify account information
print("Verifying account information...")
client.get_account_async()

Verifying account information...
[Account Details] Id:33fe434b-34ed-4da9-a3fe-0633c6d98e35, Location: eastus2


In [5]:
# Assuming you have a valid video ID to work with
# Replace with your actual video ID 
video_id = "j3xgscl9kr"  # Replace with your video ID

# Step 1: Get video insights
video_insights = client.get_video_async(video_id)
pprint(video_insights)

Searching videos in account 33fe434b-34ed-4da9-a3fe-0633c6d98e35 for video ID j3xgscl9kr.
Here are the search results: 
{'partition': None, 'description': None, 'privacyMode': 'Private', 'state': 'Processed', 'accountId': '33fe434b-34ed-4da9-a3fe-0633c6d98e35', 'id': 'j3xgscl9kr', 'name': 'DKSH Malaysia Corporate Video (2023) [JCmhHopdd08]', 'userName': 'Kyle Akepanidtaworn', 'created': '2025-03-21T07:55:41.9733333+00:00', 'isOwned': True, 'isEditable': True, 'isBase': True, 'durationInSeconds': 170, 'duration': '0:02:50.179048', 'summarizedInsights': {'name': 'DKSH Malaysia Corporate Video (2023) [JCmhHopdd08]', 'id': 'j3xgscl9kr', 'privacyMode': 'Private', 'duration': {'time': '0:02:50.179048', 'seconds': 170.2}, 'thumbnailVideoId': 'j3xgscl9kr', 'thumbnailId': 'f602d32d-8e63-4411-b98f-67080bbb44ef', 'faces': [], 'keywords': [{'isTranscript': False, 'id': 1, 'name': 'business partner', 'appearances': [{'startTime': '0:00:12.44', 'endTime': '0:00:23.28', 'startSeconds': 12.4, 'endSeco

In [6]:
# Step 2: Initialize the Azure OpenAI client using environment variables loaded above
try:
    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),  # e.g., "https://your-resource.openai.azure.com/"
        api_key=os.getenv("AZURE_OPENAI_KEY"),              # Your Azure OpenAI API key
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),  # API version string, e.g., "2023-07-01-preview"
    )

    # Confirm client initialization with a print statement
    print("Azure OpenAI client initialized successfully.")
except Exception as e:
    print(f"Error initializing Azure OpenAI client: {e}")
    sys.exit(1)  # Exit if the client couldn't be initialized

Azure OpenAI client initialized successfully.


In [7]:
# Select the GPT-4o model for text
model = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
print(model.strip())  # Print the model name for confirmation

gpt-4o


In [ ]:
# # Validate the environment variables for Azure OpenAI (Optional)
# print(os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME").split(","))  # Print the list of model names for confirmation
# print(os.getenv("AZURE_OPENAI_ENDPOINT").split(","))  # Print the list of model names for confirmation
# print(os.getenv("AZURE_OPENAI_KEY").split(","))  # Print the list of model names for confirmation
# print(os.getenv("AZURE_OPENAI_API_VERSION").split(","))  # Print the list of model names for confirmation

In [13]:
# Step 3: Extract relevant insights (e.g., transcript, keywords, topics)
transcript = " ".join([item['text'] for item in video_insights['videos'][0]['insights']['transcript']])
print("Transcript extracted successfully.")

# Prepare the summarization prompt
prompt = f"Summarize the following video transcript:\n\n{transcript}\n\nSummary:"

# Call Azure OpenAI to generate a summary
# Generate the completion  
response = client.chat.completions.create( 
    model = model,
    messages=[
        {"role": "system", "content": "You are an AI assistant that summarizes video transcripts."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=800,  
    temperature=0.7,  
    top_p=0.95,  
    frequency_penalty=0,  
    presence_penalty=0,
    stop=None,  
    stream=False
)

# Print the summary
summary = response.choices[0].message.content.strip()
print("Video Summary:\n", summary)

Transcript extracted successfully.
Video Summary:
 Dick Hairs Hitch (DKSH) is dedicated to enriching people's lives by providing access to high-quality goods and services globally, with a focus on creating sustainable value for business partners. Headquartered in Zurich, Switzerland, DKSH has a longstanding presence in Asia, spanning nearly 160 years. As a leading market expansion services provider, DKSH offers specialized services in consumer goods, healthcare, performance materials, and technology, tailored to meet the unique needs of each business partner. The company aims to exceed expectations and be a trusted growth partner, leveraging technological innovations and efficient service offerings.

DKSH's presence in Malaysia began in 1923 in Penang, with significant growth achieved through strategic acquisitions and organic strategies. It became publicly listed on the main board of Bursa Malaysia Securities in 1994. The company expanded into the fast-moving consumer goods industry i

## 